In [1]:
!pip install -q mealpy
import mealpy
mealpy.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.2/368.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 43.3 MB/s eta 0:00:00


'2.5.2'

In [2]:
import random
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from mealpy.swarm_based.SHO import OriginalSHO

In [3]:
# Datasets
# https://archive.ics.uci.edu/ml/datasets/Hepatitis

In [4]:
# Names
hepatitis_names = ['Class',
                   'AGE',
                   'SEX',
                   'STEROID',
                   'ANTIVIRALS',
                   'FATIGUE',
                   'MALAISE',
                   'ANOREXIA',
                   'LIVER BIG',
                   'LIVER FIRM',
                   'SPLEEN PALPABLE',
                   'SPIDERS',
                   'ASCITES',
                   'VARICES',
                   'BILIRUBIN',
                   'ALK PHOSPHATE',
                   'SGOT',
                   'ALBUMIN',
                   'PROTIME',
                   'HISTOLOGY']

In [5]:
# Import datasets
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/hepatitis/hepatitis.data", 
                   sep = ",",
                   names = hepatitis_names)

In [6]:
data.head()

,Class,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY
0,2,30,2,1,2,2,2,2,1,2,2,2,2,2,1.00,85,18,4.0,?,1
1,2,50,1,1,2,1,2,2,1,2,2,2,2,2,0.90,135,42,3.5,?,1
2,2,78,1,2,2,1,2,2,2,2,2,2,2,2,0.70,96,32,4.0,?,1
3,2,31,1,?,1,2,2,2,2,2,2,2,2,2,0.70,46,52,4.0,80,1
4,2,34,1,2,2,2,2,2,2,2,2,2,2,2,1.00,?,200,4.0,?,1


In [7]:
# handle missing value
data.replace(to_replace='?', value = '-1', inplace=True)

In [8]:
data.head()

,Class,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY
0,2,30,2,1,2,2,2,2,1,2,2,2,2,2,1.00,85,18,4.0,-1,1
1,2,50,1,1,2,1,2,2,1,2,2,2,2,2,0.90,135,42,3.5,-1,1
2,2,78,1,2,2,1,2,2,2,2,2,2,2,2,0.70,96,32,4.0,-1,1
3,2,31,1,-1,1,2,2,2,2,2,2,2,2,2,0.70,46,52,4.0,80,1
4,2,34,1,2,2,2,2,2,2,2,2,2,2,2,1.00,-1,200,4.0,-1,1


In [9]:
X = data.iloc[:,1:20]
y = data.iloc[:,0]

In [10]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, shuffle=True)

In [11]:
W_ENCODER = LabelEncoder()
W_ENCODER.fit(['uniform', 'distance'])
print(W_ENCODER.inverse_transform([0]))
print(W_ENCODER.inverse_transform([1]))

['distance']
['uniform']


In [14]:
def fitness_function(solution):
    N = int(solution[0])
    w_encoded = int(solution[1])
    w_decoded = W_ENCODER.inverse_transform([w_encoded])[0]

    knn = KNeighborsClassifier(n_neighbors= N, weights = w_decoded)
    knn.fit(X_train, y_train)
    y_predict = knn.predict(X_test)
    return metrics.accuracy_score(y_test, y_predict)

In [15]:
# Problem Defination
problem = {
    "fit_func": fitness_function,
    "lb": [2, 0],
    "ub": [10, 1],
    "minmax": "max",
}

In [16]:
# Termination Function
term_dict = {
    "max_early_stop": 20,
}

In [17]:
# Spotted Hyena Optimization
epoch = 100
pop_size = 50
h_factor = 5.0
N_tried = 10

model = OriginalSHO(epoch, pop_size, h_factor, N_tried)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

INFO:mealpy.swarm_based.SHO.OriginalSHO:Solving single objective optimization problem.
INFO:mealpy.swarm_based.SHO.OriginalSHO:>Problem: P, Epoch: 1, Current best: 0.7872340425531915, Global best: 0.7872340425531915, Runtime: 1.44864 seconds
INFO:mealpy.swarm_based.SHO.OriginalSHO:>Problem: P, Epoch: 2, Current best: 0.7872340425531915, Global best: 0.7872340425531915, Runtime: 1.59346 seconds
INFO:mealpy.swarm_based.SHO.OriginalSHO:>Problem: P, Epoch: 3, Current best: 0.7872340425531915, Global best: 0.7872340425531915, Runtime: 1.59909 seconds
INFO:mealpy.swarm_based.SHO.OriginalSHO:>Problem: P, Epoch: 4, Current best: 0.7872340425531915, Global best: 0.7872340425531915, Runtime: 1.74752 seconds
INFO:mealpy.swarm_based.SHO.OriginalSHO:>Problem: P, Epoch: 5, Current best: 0.7872340425531915, Global best: 0.7872340425531915, Runtime: 1.83674 seconds
INFO:mealpy.swarm_based.SHO.OriginalSHO:>Problem: P, Epoch: 6, Current best: 0.7872340425531915, Global best: 0.7872340425531915, Runtime:

Solution: [8.71889582 1.        ], Fitness: 0.7872340425531915
